# 高校数学とJulia言語 Day 5

- 城北中学校・高等学校　中学3年・高校1年
- 夏期講習会III 2025/8/24~2025/8/28
- 担当：清水団

## 本日のテーマ：確率とシミュレーション

### 5日間の学習予定
- **Day 1**：Google Colabの紹介・基本計算 ✅
- **Day 2**：関数のグラフの描画 ✅
- **Day 3**：最適化（最大・最小） ✅
- **Day 4**：データの分析 ✅
- **Day 5**：確率・シミュレーション ← 今日

今日は確率の世界をJuliaで体験し、シミュレーションで理論を確かめてみましょう！


## 確率とシミュレーションの重要性

確率とシミュレーションは現代社会の様々な分野で活用されています：

- **天気予報**：降水確率の計算
- **保険**：事故や病気のリスク評価
- **スポーツ**：勝率の予測、戦略の最適化
- **経済**：株価の変動予測、リスク管理
- **ゲーム**：確率の計算、戦略の分析
- **医療**：薬の効果や副作用の評価

シミュレーションを使うことで、実際に何度も実験を行わなくても確率的な現象を理解できます。


## 必要なパッケージの準備

確率とシミュレーションに必要なパッケージを読み込みましょう。


In [ ]:
# パッケージの読み込み
# フォント設定（日本語ラベルのため）
using Pkg
Pkg.add(url="https://github.com/ujimushi/PlotsGRBackendFontJaEmoji.jl")
using PlotsGRBackendFontJaEmoji,Plots
gr()

println("パッケージの読み込み完了！")

## 基本的な確率シミュレーション

### 問題1：コイン投げのシミュレーション

まずは最も基本的なコイン投げから始めてみましょう。理論的には表が出る確率は0.5ですが、実際はどうでしょうか？


In [ ]:
# コイン投げの関数を定義
function coin_flip()
    return rand() < 0.5 ? "表" : "裏"
end

# 10回投げてみる
println("10回のコイン投げ：")
for i in 1:10
    result = coin_flip()
    println("$(i)回目: $(result)")
end

In [ ]:
# 大量のコイン投げをシミュレーション
function simulate_coin_flips(n)
    heads_count = 0
    for i in 1:n
        if rand() < 0.5
            heads_count += 1
        end
    end
    return heads_count / n
end

# 異なる回数でシミュレーション
trials = [10, 100, 1000, 10000, 100000]
results = []

println("コイン投げシミュレーション結果：")
for n in trials
    prob = simulate_coin_flips(n)
    push!(results, prob)
    println("$(n)回投げて表が出る確率: $(round(prob, digits=4))")
end

println("理論値: 0.5")

In [ ]:
# 確率の収束を可視化
n_max = 1000
cumulative_prob = []
heads_count = 0

for i in 1:n_max
    if rand() < 0.5
        heads_count += 1
    end
    push!(cumulative_prob, heads_count / i)
end

# グラフを描画
plot(1:n_max, cumulative_prob,
     title="コイン投げの確率の収束",
     xlabel="試行回数",
     ylabel="表が出る確率",
     color=:blue, linewidth=2, label="シミュレーション結果")

# 理論値の線を追加
hline!([0.5], color=:red, linewidth=3, linestyle=:dash, label="理論値 (0.5)")

# 信頼区間を追加（参考）
hline!([0.45, 0.55], color=:gray, alpha=0.5, label="参考範囲")

## サイコロのシミュレーション

### 問題2：サイコロの目の分布

公正なサイコロなら、各目が出る確率は1/6 ≈ 0.167のはずです。実際にシミュレーションで確かめてみましょう。


In [ ]:
# サイコロを振る関数
function roll_dice()
    return rand(1:6)
end

# 大量にサイコロを振る
n_rolls = 6000
dice_results = [roll_dice() for _ in 1:n_rolls]

# 各目の出現回数を数える
counts = [sum(dice_results .== i) for i in 1:6]
probabilities = counts ./ n_rolls

println("サイコロシミュレーション結果 ($(n_rolls)回)：")
for i in 1:6
    println("$(i)の目: $(counts[i])回 (確率: $(round(probabilities[i], digits=3)))")
end
println("理論値: 各目 1/6 ≈ 0.167")

In [ ]:
# サイコロの結果をヒストグラムで表示
histogram(dice_results, bins=0.5:1:6.5,
         title="サイコロの目の分布 ($(n_rolls)回)",
         xlabel="サイコロの目",
         ylabel="出現回数",
         color=:orange,
         alpha=0.7,
         label="シミュレーション結果")

# 理論値の線を追加
hline!([n_rolls/6], color=:red, linewidth=3, linestyle=:dash, label="理論値 ($(round(n_rolls/6)))")

### 問題3：2つのサイコロの和

2つのサイコロを振って、その和を調べてみましょう。どの数字が最も出やすいでしょうか？


In [ ]:
# 2つのサイコロの和をシミュレーション
n_rolls = 10000
sums = []

for _ in 1:n_rolls
    dice1 = roll_dice()
    dice2 = roll_dice()
    push!(sums, dice1 + dice2)
end

# 理論的な確率を計算
theoretical_probs = Dict(
    2 => 1/36, 3 => 2/36, 4 => 3/36, 5 => 4/36, 6 => 5/36, 7 => 6/36,
    8 => 5/36, 9 => 4/36, 10 => 3/36, 11 => 2/36, 12 => 1/36
)

println("2つのサイコロの和の分布：")
for s in 2:12
    count = sum(sums .== s)
    simulated_prob = count / n_rolls
    theoretical_prob = theoretical_probs[s]
    println("和=$(s): 実験=$(round(simulated_prob, digits=3)), 理論=$(round(theoretical_prob, digits=3))")
end

In [ ]:
# 2つのサイコロの和のヒストグラム
histogram(sums, bins=1.5:1:12.5,
         title="2つのサイコロの和の分布",
         xlabel="サイコロの和",
         ylabel="出現回数",
         color=:green,
         alpha=0.7,
         normalize=true,
         label="シミュレーション結果")

# 理論値をプロット
theoretical_x = 2:12
theoretical_y = [theoretical_probs[s] for s in theoretical_x]
plot!(theoretical_x, theoretical_y,
      color=:red, linewidth=3, marker=:circle, markersize=6,
      label="理論値")

## 誕生日のパラドックス

### 問題4：クラスに同じ誕生日の人がいる確率

「30人のクラスで、同じ誕生日の人が2人以上いる確率は？」
直感では低そうですが、実際は驚くべき結果になります。


In [ ]:
# 誕生日の重複をチェックする関数
function has_birthday_collision(n_people)
    birthdays = rand(1:365, n_people)  # 1-365の誕生日をランダムに生成
    return length(unique(birthdays)) < n_people  # 重複があればtrue
end

# 異なる人数での誕生日パラドックスをシミュレーション
function birthday_paradox_simulation(n_people, n_trials=10000)
    collisions = 0
    for _ in 1:n_trials
        if has_birthday_collision(n_people)
            collisions += 1
        end
    end
    return collisions / n_trials
end

# 様々な人数で実験
people_counts = [10, 15, 20, 23, 25, 30, 35, 40, 50]
probabilities = []

println("誕生日パラドックスのシミュレーション結果：")
for n in people_counts
    prob = birthday_paradox_simulation(n)
    push!(probabilities, prob)
    println("$(n)人: $(round(prob, digits=3))")
end

In [ ]:
# 理論値の計算
function birthday_theory(n)
    if n > 365
        return 1.0
    end

    prob_no_collision = 1.0
    for i in 0:(n-1)
        prob_no_collision *= (365 - i) / 365
    end

    return 1 - prob_no_collision
end

# 理論値を計算
theoretical_probs = [birthday_theory(n) for n in people_counts]

println("\n理論値との比較：")
for i in 1:length(people_counts)
    n = people_counts[i]
    sim = probabilities[i]
    theory = theoretical_probs[i]
    println("$(n)人: シミュレーション=$(round(sim, digits=3)), 理論=$(round(theory, digits=3))")
end

In [ ]:
# 誕生日パラドックスの可視化
plot(people_counts, probabilities,
     marker=:circle, markersize=6, linewidth=2, color=:blue,
     title="誕生日パラドックス",
     xlabel="クラスの人数",
     ylabel="同じ誕生日の人がいる確率",
     label="シミュレーション結果")

plot!(people_counts, theoretical_probs,
      linewidth=3, color=:red, linestyle=:dash,
      label="理論値")

# 50%ラインを追加
hline!([0.5], color=:gray, linestyle=:dot, alpha=0.7, label="50%ライン")

# 23人のポイントを強調
scatter!([23], [birthday_theory(23)],
         markersize=10, color=:orange,
         label="23人 ($(round(birthday_theory(23)*100, digits=1))%)")

## モンテカルロ法による円周率の計算

### 問題5：ランダムな点を使って円周率を求める

モンテカルロ法は、ランダムな点を使って数値を近似する手法です。円周率πを計算してみましょう。


In [ ]:
# モンテカルロ法でπを計算
function estimate_pi(n_points)
    inside_circle = 0

    for _ in 1:n_points
        x = rand() * 2 - 1  # -1から1の範囲
        y = rand() * 2 - 1  # -1から1の範囲

        # 原点からの距離が1以下なら円の内部
        if x^2 + y^2 <= 1
            inside_circle += 1
        end
    end

    # π ≈ 4 × (円内の点の数) / (全体の点の数)
    return 4 * inside_circle / n_points
end

# 異なる点数でπを推定
point_counts = [100, 1000, 10000, 100000, 1000000]
pi_estimates = []

println("モンテカルロ法による円周率の推定：")
for n in point_counts
    pi_est = estimate_pi(n)
    push!(pi_estimates, pi_est)
    error = abs(pi_est - π)
    println("$(n)点: π ≈ $(round(pi_est, digits=4)), 誤差: $(round(error, digits=4))")
end

println("真の値: π = $(round(π, digits=6))")

In [ ]:
# モンテカルロ法の可視化（少数の点で）
n_points = 1000
x_points = []
y_points = []
colors = []

for _ in 1:n_points
    x = rand() * 2 - 1
    y = rand() * 2 - 1
    push!(x_points, x)
    push!(y_points, y)

    # 円の内部なら青、外部なら赤
    if x^2 + y^2 <= 1
        push!(colors, :blue)
    else
        push!(colors, :red)
    end
end

# 散布図をプロット
scatter(x_points, y_points,
        color=colors, markersize=2, alpha=0.6,
        title="モンテカルロ法による円周率計算",
        xlabel="x", ylabel="y",
        aspect_ratio=:equal,
        label="ランダム点")

# 単位円を描画
θ = 0:0.01:2π
circle_x = cos.(θ)
circle_y = sin.(θ)
plot!(circle_x, circle_y, color=:black, linewidth=3, label="単位円")

# 正方形の枠を描画
plot!([-1, 1, 1, -1, -1], [-1, -1, 1, 1, -1],
      color=:black, linewidth=2, linestyle=:dash, label="正方形")

In [ ]:
# πの推定値の収束を可視化
plot(point_counts, pi_estimates,
     marker=:circle, markersize=6, linewidth=2, color=:green,
     title="モンテカルロ法によるπの収束",
     xlabel="使用した点の数",
     ylabel="πの推定値",
     xscale=:log10,
     label="推定値")

# 真のπの値を追加
hline!([π], color=:red, linewidth=3, linestyle=:dash, label="真の値 π")

# 信頼区間を追加
hline!([π-0.1, π+0.1], color=:gray, alpha=0.5, label="±0.1の範囲")

## Day 5 の演習問題

以下の問題に取り組んで、確率とシミュレーションの理解を深めましょう。

### 問題1: ジャンケンの確率

ジャンケンで「あいこ」になる確率をシミュレーションで求めてください。

1. ジャンケン関数を作成（グー=1, チョキ=2, パー=3）
2. 10000回のジャンケンをシミュレーション
3. あいこになる確率を計算
4. 理論値（1/3）と比較

### 問題2: 3つのサイコロの最大値

3つのサイコロを同時に振って、最大の目が6になる確率を求めてください。

1. 3つのサイコロを振るシミュレーション関数を作成
2. 最大値が6になる場合をカウント
3. ヒストグラムで最大値の分布を可視化
4. 理論値と比較（ヒント：最大値が6になるのは、少なくとも1つが6の場合）

### 問題3: 自由課題

以下のいずれかの問題を選んで解いてください：

**A. モンティ・ホール問題**
- 3つの扉のうち1つに当たりがある
- 1つ選んだ後、ホストがハズレの扉を1つ開ける
- 選択を変更する戦略と変更しない戦略の勝率を比較

**B. ランダムウォーク**
- 原点から始まって、毎ステップ±1の移動をランダムに行う
- 1000ステップ後の位置の分布を調べる
- 複数回のシミュレーションで軌跡を可視化

**C. オリジナル問題**
- 自分で面白い確率問題を考えて解く
- 例：宝くじ、スポーツの勝敗予測、ゲームの確率など


## 解答欄

以下のセルに解答を記入してください。


### 問題1の解答：ジャンケンの確率

In [ ]:
# 問題1: ジャンケンの確率
# ジャンケンで「あいこ」になる確率をシミュレーションで求めてください。
# 1. ジャンケン関数を作成（グー=1, チョキ=2, パー=3）


In [ ]:
# 2. 10000回のジャンケンをシミュレーション


In [ ]:
# 3. あいこになる確率を計算


In [ ]:
# 4. 理論値（1/3）と比較


### 問題2の解答：3つのサイコロの最大値

In [ ]:
# 問題2: 3つのサイコロの最大値
# 3つのサイコロを同時に振って、最大の目が6になる確率を求めてください。

# 1. 3つのサイコロを振るシミュレーション関数を作成


In [ ]:
# 2. 最大値が6になる場合をカウント


In [ ]:
# 3. ヒストグラムで最大値の分布を可視化



In [ ]:
# 4. 理論値と比較（ヒント：最大値が6になるのは、少なくとも1つが6の場合）


### 問題3の解答：自由課題

In [ ]:
# 問題3: 自由課題
# ここに解答を書いてください

# **A. モンティ・ホール問題**
# - 3つの扉のうち1つに当たりがある
# - 1つ選んだ後、ホストがハズレの扉を1つ開ける
# - 選択を変更する戦略と変更しない戦略の勝率を比較





In [ ]:
# **B. ランダムウォーク**
# - 原点から始まって、毎ステップ±1の移動をランダムに行う
# - 1000ステップ後の位置の分布を調べる
# - 複数回のシミュレーションで軌跡を可視化


In [ ]:
# **C. オリジナル問題**
# - 自分で面白い確率問題を考えて解く
# - 例：宝くじ、スポーツの勝敗予測、ゲームの確率など

## まとめ

5日間の「高校数学とJulia言語」講習会の最終日、お疲れさまでした！

### 今日学習した内容

#### 確率の基本
- **コイン投げ**：基本的な確率シミュレーション
- **サイコロ**：等確率事象の検証
- **確率の収束**：大数の法則の実体験

#### 応用的な確率問題
- **2つのサイコロの和**：複合事象の分析
- **誕生日パラドックス**：直感に反する確率現象
- **モンテカルロ法**：ランダムサンプリングによる数値計算

#### シミュレーションの威力
- 理論計算が困難な問題でも近似解が得られる
- 直感と理論の違いを実際に確かめられる
- 大量のデータで統計的な性質を理解できる

### 5日間の総復習

#### **Day 1**: Julia言語の基礎
- Google Colabの使い方
- 基本的な計算と数学関数
- 変数と数式の計算

#### **Day 2**: 関数とグラフ
- 関数の定義方法
- 様々な関数のグラフ描画
- 複数のグラフの比較と分析

#### **Day 3**: 最適化
- 関数の最大・最小値の数値的探索
- グラフによる視覚的理解
- 制約条件付きの最適化問題

#### **Day 4**: データ分析
- 統計量の計算（平均、分散、相関など）
- ヒストグラムと散布図による可視化
- 回帰分析と関係性の理解

#### **Day 5**: 確率とシミュレーション
- 確率現象のシミュレーション
- 理論値と実験値の比較
- モンテカルロ法による数値計算

### プログラミングの学習効果

この5日間で、皆さんは：
- **数学の理論**をプログラムで実践的に確認
- **データの可視化**で直感的な理解を深化
- **シミュレーション**で複雑な現象を分析
- **問題解決**のための新しいツールを習得

### 今後の学習に向けて

プログラミングと数学の組み合わせは、今後の学習や将来の仕事で大きな武器になります：

- **大学数学**：微積分、線形代数、統計学の理解が深まる
- **理系分野**：物理、化学、生物学での数値実験
- **文系分野**：経済学、心理学でのデータ分析
- **実社会**：AI、データサイエンス、金融工学など

### 最終メッセージ

数学は「考える力」を、プログラミングは「実現する力」を育てます。この2つを組み合わせることで、皆さんの可能性は無限に広がります。

今回学んだ技術を使って、ぜひ自分だけの面白いプロジェクトに挑戦してみてください！

### 提出について

このノートブックに解答を記入し、保存してからGoogle Classroomに提出してください。5日間の学習の集大成として、問題3では創造性溢れる解答を期待しています！

### 参考資料

- Julia Documentation: https://docs.julialang.org/
- Julia Statistics: https://docs.julialang.org/en/v1/stdlib/Statistics/
- Julia Random: https://docs.julialang.org/en/v1/stdlib/Random/
- 確率・統計の参考書
- プログラミング学習サイト

## お疲れさまでした！🎉

5日間の講習会、本当にお疲れさまでした。皆さんの今後の学習と成長を心から応援しています！
